In [ ]:
import ee
import eemont  # noqa: F401
import geemap
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from bisonlab.data import s2_sr_harmonized

In [ ]:
# ee.Authenticate()
ee.Initialize()

In [ ]:
geometry = ee.Geometry.Point([-108.8028753344979, 43.2067524527917]).buffer(10000);
dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.bounds(geometry))
trueColor = dataset.select(['R', 'G', 'B'])
trueColorVis = {
  "min": 0.0,
  "max": 255.0,
}
m = geemap.Map()
m.addLayer(trueColor, trueColorVis, 'True Color')
m.centerObject(geometry, 15)
m

In [ ]:
buffalo_expansion = ee.FeatureCollection(
    "projects/colinhill/assets/bison-lab/Buffalo_Expansion_polygons"
)

In [ ]:
parcels = buffalo_expansion.filter(
    ee.Filter.inList("Name", ["Shoshone Tribe", "Hellyer Tribal Lease"])
)
# aoi = ee.Geometry.Point(-108.79519868020131, 43.20280898430678)

In [ ]:
start_date = "2017-03-28"
end_date = "2022-11-01"
indices = ["NDVI", "EVI", "SAVI", "GVMI", "MNDWI", "NDDI"]

In [ ]:
s2 = s2_sr_harmonized(parcels, start_date, end_date).spectralIndices(indices)

In [ ]:
ts = s2.getTimeSeriesByRegions(
    reducer=ee.Reducer.mean(),
    collection=parcels,
    bands=None,
    scale=10,
    dateColumn="date",
    naValue=None,
)

In [ ]:
df = geemap.ee_to_pandas(ts)

In [ ]:
df = df.drop(
    columns=[
        "visibility",
        "descriptio",
        "tessellate",
        "extrude",
        "icon",
        "end",
        "begin",
        "timestamp",
        "altitudeMo",
    ]
)

In [ ]:
filename = "s2_sr_indices_mean.csv"
# df.to_csv(filename)
df = pd.read_csv(filename)

### Drop bad data

In [ ]:
img = s2.closest("2021-11-08").first()

In [ ]:
m = geemap.Map()
m.addLayer(img, {"min": 0, "max": 0.3, "bands": ["B4", "B3", "B2"]}, "s2")
# m.addLayer(img, {"min": -1, "max": 1, "palette": cm.palettes.ndvi}, "")
m.centerObject(parcels, 12)
m

In [ ]:
# df[df['NDVI'] > 0.8]
# df[df['NDDI'] < -1.75e8]

In [ ]:
df = df.set_index("date")
df = df.drop("2021-11-08T18:22:16").reset_index()

### Plot

In [ ]:
a = df.melt(id_vars=["date", "Name"], value_vars=indices)

In [ ]:
g = sns.relplot(
    data=a,
    x="date",
    y="value",
    hue="Name",
    col="variable",
    col_wrap=1,
    kind="line",
    height=4,
    aspect=2,
    facet_kws=dict(sharey=False),
)
for ax in g.axes:
    ax.xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.lineplot(data=df, x="date", y="NDVI", hue="Name", ax=ax)
ax.xaxis.set_major_locator(mdates.MonthLocator())
fig.autofmt_xdate()